In [ ]:
%%capture
!apt-get install tesseract-ocr
!apt-get install pciutils
!pip install pytesseract Pillow discord langchain_community colab-xterm

#@markdown <br><center><h3><font color=white><b>🛠️ INSTALL DEPENDENCIES 🛠️</b></h3>

In [ ]:
#@markdown <br><center><h2><font color=white><b>📥 Download Ollama Models 📥</b></h2>
#@markdown <br><center><h4><font color="white">```ollama serve & ollama run llama3```</center></br></h4>
#@markdown <br><center><font color=red>NOTE:<font color=white> You can run any model you want. Just go to the link below and try whichever model you prefer. Don't forget to update the model in the code below.</br>
#@markdown <br><center><font color=white>MORE MODELS : https://ollama.com/library</br>

%load_ext colabxterm
%xterm
!curl -faSL https://ollama.ai/install.sh | sh

In [ ]:
#@markdown <br><center><h3><font color=white><b>🔄 RUN 🤖 BOT 🔄</b></h3>
BOT_TOKEN = "" #@param {type:"string"}
CHANNEL_ID = "" #@param {type:"string"}
CHANNEL_ID = int(CHANNEL_ID)

#@markdown <br><center><a href="https://discordgsm.com/guide/how-to-get-a-discord-bot-token" target="_blank">GET DISCORD BOT TOKEN</a>
#@markdown <br><center><a href="https://support.discord.com/hc/en-us/articles/206346498-Where-can-I-find-my-User-Server-Message-ID" target="_blank">GET CHANNEl ID</a>

#@markdown <br><center><h4><font color=white><b> DON'T FORGET TO RUN THE LOCAL SCRIPT </b></h3>

import json
import discord
from io import BytesIO
import pytesseract
from PIL import Image
import nest_asyncio
import sys
import os
from langchain_community.llms import Ollama

# Function to apply nest_asyncio patch if necessary
def apply_nest_asyncio_patch():
    try:
        nest_asyncio.apply()
    except Exception as e:
        print(f"Error applying nest_asyncio patch: {e}")
        sys.exit(1)

def ask_model(prompt):
  optimised_prompt = f"only reply with correct answer and ignore explanation, also make sure to indend the code, plesae execute code backend and only provide me both correct output and correct option from the question, also make sure to provide answers in 2 lines, 1st line backend execute output, 2nd line correct option in text only \n\n {prompt}"
  llm = Ollama(model='llama3')
  response = llm.invoke(optimised_prompt)
  return response

def extract_text_from_image(image_file):
    try:
        img = Image.open(image_file)
        text = pytesseract.image_to_string(img)
        return text
    except Exception as e:
        return f"Error occurred: {e}"

def main():
    apply_nest_asyncio_patch()
    TOKEN = BOT_TOKEN

    intents = discord.Intents.default()
    intents.messages = True
    intents.message_content = True

    client = discord.Client(intents=intents)

    @client.event
    async def on_ready():
        print(f'Logged in as {client.user}!')
        await client.change_presence(activity=discord.Game(name="Made by JuniorAlive"))

    @client.event
    async def on_message(message):
        if message.channel.id != CHANNEL_ID:  # Your channel ID
            return

        if message.attachments:
            for attachment in message.attachments:
                response = await attachment.read()
                if any(attachment.filename.endswith(ext) for ext in ['.png', '.jpg', '.jpeg', '.gif', '.bmp']):
                    image_to_text = extract_text_from_image(BytesIO(response))
                    extracted_text = image_to_text if image_to_text else "Failed to extract text"
                    response_message = ask_model(extracted_text)
                    if len(response_message) > 1900:
                        with open("output.txt", "w", encoding="utf-8") as text_file:
                            text_file.write(response_message)
                        with open("output.txt", "rb") as text_file:
                            await message.reply(file=discord.File(text_file, "output.txt"))
                    else:
                        await message.reply("```" + response_message.strip()[:1990] + "```")
                else:
                    await message.reply("Unsupported file type.")

    client.run(TOKEN)

if __name__ == "__main__":
    main()
